# Criptografia simétrica com Python

É fundamental o uso bem aplicado de criptografia simétrica para garantir a confidencialidade em sistemas modernos. Com esse roteiro você aprenderá a fazer isso utilizando Python.
<br>
<br>
Para isso, utilizaremos a biblioteca **[cryptography](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/)**, que é a biblioteca mais utilizada para realizar esse tipo de operação. Também seguiremos as recomendações de algoritmos e modos de operação préviamente expostas (AES-128 e CTR).

## Conhecendo a biblioteca

A biblioteca possui várias funcionalidades como criptografia simétrica, criptografia assimétrica, hashing, HMAC, etc. No nosso caso iremos utilizar apenas a criptografia simétrica.
<br>
<br>
Para usar todo esse ferramental, precisamos instalar a biblioteca.

In [ ]:
!pip install cryptography

## Lidando com chaves

Antes de começarmos a cifrar alguma coisa, primeiro precisamos entender como gerar chaves para utiliza-las com a biblioteca.
<br>
<br>
Se a chave gerada não for aleatória o suficiente e tiver alguma correlação com ser dependente do tempo, dependente da máquina ou uma palavra do dicionário, por exemplo, ela se torna vulnerável. Para evitar palavras conhecidas, vamos utilizar um gerador aleatório de bytes ao invés de gerá-las manualmente (o que também seria pouco escalável).
<br>
<br>
Existe algumas formas de se gerar chaves aleatórias, por exemplo usando ```random.randbytes(bytes_size: int)``` ou  ```os.urandom(bytes_size: int)```, porém elas não são seguras pois utilizam informações da máquina, como timestamp para gerar os valores.
<br>
<br>
A forma mais aconselhadas de se gerar numeros de forma segura é utilizando a função ```.token_bytes(bytes_size: int)``` da biblioteca ```secrets```. Como vamos utilizar o gerador para obter uma chave secreta para usar no AES-128 e um nonce para o modo de operação counter, precisamos de uma chave secreta de 128 bits e também de um nonce de 128 bits.

In [ ]:
import secrets

# lembraando que 1 byte = 8 bits, então 16 bytes = 128 bits
key = secrets.token_bytes(16)
nonce = secrets.token_bytes(16)

## Configurando o Cipher

Bem, agora que já temos nossa chave secreta e o nosso nonce, podemos começar a cifrar dados. Para isso, instaciamos um objeto da classe `Cipher` e passamos como parâmetro o algoritmo que será utilizado (AES) e modo de operação (counter).
<br>
<br>
**obs:** A variação do algoritmo (no caso do AES) pode estar explicito ou definido pelo tamanho da chave passada. No nosso caso, definiremos pela chave de 128 bits passada no construtor.

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))

A classe `Cipher` não cifra nem decifra nada, apenas armazena as informações necessárias e retorna duas interfaces, o encryptor e o decryptor. E é com elas que realizamos as operações.


## Cifrando uma mensagem

Para enfim cifrar uma mensagem, precisamos criar uma interface `encryptor` e a partir dela cifrar nossos dados utilizando o método `.update(plain_text: bytes)`, que retorna o texto cifrado. Quando terminarmos de cifrar o que desejamos, devemos chamar `.finalize()` para retornar algum dado que ainda não tenha sido retornado.

In [ ]:
encryptor = cipher.encryptor()

texto_plano = "O segredo do fotografo ainda nao foi revelado."
cipher_text = encryptor.update(texto_plano.encode()) + encryptor.finalize()

print(cipher_text)

Uma coisa que deve ser observada é que a API foi montada de forma que facilita o processamento de dados grandes em pedaços menores. Porém cuidado quando for fazer paralelismo aqui, pois existem modos de operação que não suportam cifrar de forma paralela.

In [ ]:
encryptor = cipher.encryptor()
ct = b''

with open('file.txt', 'w') as file:
  file.write("Esse arquivo é para mostrar um exemplo na cadeira de sistemas distribuídos.")

with open('file.txt', 'rb') as file:
  chunk = file.read(5) # para ler de 5 em 5 bytes

  while chunk:
    ct += encryptor.update(chunk)
    chunk = file.read(5)

ct += encryptor.finalize()

**obs:** Padding não é adicionado automaticamente, caso você tenha problema do dado não ser ser um múltiplo do tamanho do bloco, adicione o [padding](https://cryptography.io/en/latest/hazmat/primitives/padding/#cryptography.hazmat.primitives.padding.PKCS7).

## Decifrando a mensagem

Decifrar dados usando essa biblioteca é muito semelhante, precisamos criar uma interface `decryptor` e a partir dela cifrar nossos dados utilizando o método `.update(plain_text: bytes)`, que retorna o texto plano. Quando terminarmos de de
cifrar o que desejamos, devemos chamar `.finalize()` para retornar algum dado que ainda não tenha sido retornado.

In [ ]:
decryptor = cipher.decryptor()
texto = decryptor.update(cipher_text) + decryptor.finalize() # ainda está em bytes

print(texto.decode())

O segredo do fotografo ainda nao foi revelado.


**obs:** Da mesma forma que partimos o processo de cifrar de um dado grande, podemos fazer o de decifrar.

# Atividade

Para praticar o que foi aprendido, seu objetivo é puxar os dados de um
arquivo .csv criptografada, descriptografar, fazer a soma dos números entre colunas, cifrar o resultado e enviar de volta [nesse form](). Você deve fazer isso usando o algoritmo, modo de operação, segredos e o arquivo especificado [aqui]().
<br>
<br>
Você deve retornar a base de dados e o código usado. Você pode utilizar qualquer linguagem que desejar.
<br>
<br>
Boa sorte!
<br>
<br>
*Para versão do colab disponibilizada no GitHub eu removi os links. Posteriormente eu vou gerar a chave, o csv, etc especificamente para essa versão do colab.*